In [ ]:
#| default_exp models.alternatingtransformer

# Alternating Transformer

Vanilla Transformer with alternating spatial and temporal attention.

In [ ]:
#| export
import numpy as np
from typing import Optional
import torch
import torch.nn as nn

from neuralforecast.common._modules import (
    TransEncoderLayer, TransEncoder,
    TransDecoderLayer, TransDecoder,
    DataEmbedding, AttentionLayer, FullAttention,
)
from neuralforecast.models.vanillatransformer import VanillaTransformer
from neuralforecast.common._base_model import BaseModel
from neuralforecast.losses.pytorch import MAE


In [ ]:
#| hide
import logging
import warnings
from fastcore.test import test_eq
from nbdev.showdoc import show_doc
from neuralforecast.common._model_checks import check_model


In [ ]:
#| export
class _AltEncoderLayer(nn.Module):
    """Encoder layer with spatial or temporal attention."""

    def __init__(self, mode, hidden_size, n_head, conv_hidden_size, dropout, activation):
        super().__init__()
        self.mode = mode
        self.layer = TransEncoderLayer(
            AttentionLayer(
                FullAttention(mask_flag=False, attention_dropout=dropout, output_attention=False),
                hidden_size,
                n_head,
            ),
            hidden_size,
            conv_hidden_size,
            dropout=dropout,
            activation=activation,
        )

    def forward(self, x):
        # x: [B, L, N, D]
        B, L, N, D = x.shape
        if self.mode == 'temporal':
            x_r = x.reshape(B * L, N, D)
            out, _ = self.layer(x_r, attn_mask=None)
            x = out.reshape(B, L, N, D)
        else:  # spatial
            x_r = x.permute(0, 2, 1, 3).reshape(B * N, L, D)
            out, _ = self.layer(x_r, attn_mask=None)
            x = out.reshape(B, N, L, D).permute(0, 2, 1, 3)
        return x


class _AltEncoder(nn.Module):
    def __init__(self, num_layers, hidden_size, n_head, conv_hidden_size, dropout, activation):
        super().__init__()
        self.layers = nn.ModuleList([
            _AltEncoderLayer('temporal' if i % 2 == 0 else 'spatial', hidden_size, n_head, conv_hidden_size, dropout, activation)
            for i in range(num_layers)
        ])
        self.norm = nn.LayerNorm(hidden_size)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.norm(x), None


class _AltDecoderLayer(nn.Module):
    def __init__(self, mode, hidden_size, n_head, conv_hidden_size, dropout, activation):
        super().__init__()
        self.mode = mode
        self.layer = TransDecoderLayer(
            AttentionLayer(
                FullAttention(mask_flag=True, attention_dropout=dropout, output_attention=False),
                hidden_size,
                n_head,
            ),
            AttentionLayer(
                FullAttention(mask_flag=False, attention_dropout=dropout, output_attention=False),
                hidden_size,
                n_head,
            ),
            hidden_size,
            conv_hidden_size,
            dropout=dropout,
            activation=activation,
        )

    def forward(self, x, cross, x_mask=None, cross_mask=None):
        B, L, N, D = x.shape
        enc_len = cross.shape[1]
        if self.mode == 'temporal':
            x_r = x.reshape(B * L, N, D)
            cross_r = cross.reshape(B * enc_len, N, D)
            out = self.layer(x_r, cross_r, x_mask=x_mask, cross_mask=cross_mask)
            x = out.reshape(B, L, N, D)
        else:
            x_r = x.permute(0, 2, 1, 3).reshape(B * N, L, D)
            cross_r = cross.permute(0, 2, 1, 3).reshape(B * N, enc_len, D)
            out = self.layer(x_r, cross_r, x_mask=x_mask, cross_mask=cross_mask)
            x = out.reshape(B, N, L, D).permute(0, 2, 1, 3)
        return x


class _AltDecoder(nn.Module):
    def __init__(self, num_layers, hidden_size, n_head, conv_hidden_size, dropout, activation, projection):
        super().__init__()
        self.layers = nn.ModuleList([
            _AltDecoderLayer('temporal' if i % 2 == 0 else 'spatial', hidden_size, n_head, conv_hidden_size, dropout, activation)
            for i in range(num_layers)
        ])
        self.norm = nn.LayerNorm(hidden_size)
        self.projection = projection

    def forward(self, x, cross, x_mask=None, cross_mask=None):
        for layer in self.layers:
            x = layer(x, cross, x_mask=x_mask, cross_mask=cross_mask)
        if self.norm is not None:
            x = self.norm(x)
        if self.projection is not None:
            x = self.projection(x)
        return x

class AlternatingTransformer(VanillaTransformer):
    """Vanilla Transformer with alternating spatial and temporal attention."""

    EXOGENOUS_FUTR = True
    EXOGENOUS_HIST = False
    EXOGENOUS_STAT = False
    MULTIVARIATE = True
    RECURRENT = False

    def __init__(
        self,
        h: int,
        input_size: int,
        n_series: int,
        stat_exog_list=None,
        hist_exog_list=None,
        futr_exog_list=None,
        exclude_insample_y: bool = False,
        decoder_input_size_multiplier: float = 0.5,
        hidden_size: int = 128,
        dropout: float = 0.05,
        n_head: int = 4,
        conv_hidden_size: int = 32,
        activation: str = "gelu",
        encoder_layers: int = 2,
        decoder_layers: int = 1,
        loss=MAE(),
        valid_loss=None,
        max_steps: int = 5000,
        learning_rate: float = 1e-4,
        num_lr_decays: int = -1,
        early_stop_patience_steps: int = -1,
        val_check_steps: int = 100,
        batch_size: int = 32,
        valid_batch_size: Optional[int] = None,
        windows_batch_size=1024,
        inference_windows_batch_size: int = 1024,
        start_padding_enabled=False,
        step_size: int = 1,
        scaler_type: str = "identity",
        random_seed: int = 1,
        drop_last_loader: bool = False,
        alias: Optional[str] = None,
        optimizer=None,
        optimizer_kwargs=None,
        lr_scheduler=None,
        lr_scheduler_kwargs=None,
        dataloader_kwargs=None,
        **trainer_kwargs,
    ):
        BaseModel.__init__(
            self,
            h=h,
            input_size=input_size,
            n_series=n_series,
            stat_exog_list=stat_exog_list,
            hist_exog_list=hist_exog_list,
            futr_exog_list=futr_exog_list,
            exclude_insample_y=exclude_insample_y,
            loss=loss,
            valid_loss=valid_loss,
            max_steps=max_steps,
            learning_rate=learning_rate,
            num_lr_decays=num_lr_decays,
            early_stop_patience_steps=early_stop_patience_steps,
            val_check_steps=val_check_steps,
            batch_size=batch_size,
            valid_batch_size=valid_batch_size,
            windows_batch_size=windows_batch_size,
            inference_windows_batch_size=inference_windows_batch_size,
            start_padding_enabled=start_padding_enabled,
            step_size=step_size,
            scaler_type=scaler_type,
            drop_last_loader=drop_last_loader,
            alias=alias,
            random_seed=random_seed,
            optimizer=optimizer,
            optimizer_kwargs=optimizer_kwargs,
            lr_scheduler=lr_scheduler,
            lr_scheduler_kwargs=lr_scheduler_kwargs,
            dataloader_kwargs=dataloader_kwargs,
            **trainer_kwargs,
        )

        self.label_len = int(np.ceil(input_size * decoder_input_size_multiplier))
        if (self.label_len >= input_size) or (self.label_len <= 0):
            raise Exception(
                f"Check decoder_input_size_multiplier={decoder_input_size_multiplier}, range (0,1)"
            )

        if activation not in ["relu", "gelu"]:
            raise Exception(f"Check activation={activation}")

        self.c_out = self.loss.outputsize_multiplier
        self.enc_in = 1
        self.dec_in = 1
        self.n_series = n_series

        self.enc_embedding = DataEmbedding(
            c_in=self.enc_in,
            exog_input_size=self.futr_exog_size,
            hidden_size=hidden_size,
            pos_embedding=True,
            dropout=dropout,
        )
        self.dec_embedding = DataEmbedding(
            self.dec_in,
            exog_input_size=self.futr_exog_size,
            hidden_size=hidden_size,
            pos_embedding=True,
            dropout=dropout,
        )

        self.encoder = _AltEncoder(
            num_layers=encoder_layers,
            hidden_size=hidden_size,
            n_head=n_head,
            conv_hidden_size=conv_hidden_size,
            dropout=dropout,
            activation=activation,
        )
        self.decoder = _AltDecoder(
            num_layers=decoder_layers,
            hidden_size=hidden_size,
            n_head=n_head,
            conv_hidden_size=conv_hidden_size,
            dropout=dropout,
            activation=activation,
            projection=nn.Linear(hidden_size, self.c_out, bias=True),
        )

    def forward(self, windows_batch):
        insample_y = windows_batch["insample_y"]  # [B, L, N]
        futr_exog = windows_batch["futr_exog"]  # [B, F, L+h, N]
        B, L, N = insample_y.shape

        insample_y = insample_y.permute(0, 2, 1).reshape(B * N, L, 1)
        if self.futr_exog_size > 0:
            futr = futr_exog.permute(0, 3, 2, 1).reshape(B * N, L + self.h, self.futr_exog_size)
            x_mark_enc = futr[:, : L, :]
            x_mark_dec = futr[:, -(self.label_len + self.h) :, :]
        else:
            x_mark_enc = None
            x_mark_dec = None

        x_dec = torch.zeros(size=(B * N, self.h, 1), device=insample_y.device)
        x_dec = torch.cat([insample_y[:, -self.label_len :, :], x_dec], dim=1)

        enc_out = self.enc_embedding(insample_y, x_mark_enc)
        enc_out = enc_out.view(B, N, L, -1).permute(0, 2, 1, 3)
        enc_out, _ = self.encoder(enc_out)

        dec_out = self.dec_embedding(x_dec, x_mark_dec)
        dec_out = dec_out.view(B, N, self.label_len + self.h, -1).permute(0, 2, 1, 3)
        dec_out = self.decoder(dec_out, enc_out)

        forecast = dec_out[:, -self.h :, :, :]
        forecast = forecast.reshape(B, self.h, N * self.c_out)
        return forecast